In [2]:
%pwd

'/home/giakhang/dev/pose_sandbox'

In [1]:
import cv2
import numpy as np
from mmdeploy_runtime import Detector, PoseDetector

In [3]:
def visualize(frame, keypoints, filename, thr=0.5, resize=1280):
    skeleton = [
        [0, 5], [5, 7], [7, 92], 
        [92, 93], [93, 94], [94, 95], [95, 96],
        [92, 97], [97, 98], [98, 99], [99, 100],
        [101, 102], [102, 103], [103, 104],
        [105, 106], [106, 107], [107, 108],
        [92, 109], [109, 110], [110, 111], [111, 112],
        [93, 97], [97, 101], [101, 105], [105, 109]
    ]

    palette = [
        # Body colors
        (255, 128, 0), (255, 153, 51), (255, 178, 102), (230, 230, 0), 
        (255, 153, 255), (153, 204, 255), (255, 102, 255), (255, 51, 255),
        (102, 178, 255), (51, 153, 255), (255, 153, 153), (255, 102, 102), 
        (255, 51, 51), (153, 255, 153), (102, 255, 102), (51, 255, 51),
        (0, 255, 0), (0, 0, 255), (255, 0, 0),
        # Face colors
        (255, 255, 0), (255, 255, 102), (255, 255, 153), (255, 255, 204),
        # Right Hand colors (21 keypoints)
        (153, 0, 255), (178, 102, 255), (204, 153, 255), (230, 204, 255),
        (255, 204, 255), (204, 102, 255), (178, 51, 255), (153, 51, 255),
        (128, 51, 255), (102, 51, 255), (51, 51, 255), (102, 102, 255),
        (153, 153, 255), (204, 204, 255), (230, 230, 255), (255, 230, 255),
        (255, 153, 230), (255, 102, 230), (255, 51, 230), (255, 0, 230),
        # Left Hand colors (21 keypoints)
        (0, 153, 255), (51, 178, 255), (102, 204, 255), (153, 230, 255),
        (204, 255, 255), (51, 255, 204), (51, 255, 153), (51, 255, 102),
        (51, 255, 51), (102, 255, 102), (153, 255,153), (153, 255,204),
        (153, 255,255), (204, 255,255), (255, 255,204), (255, 255,153),
        (255, 255,102), (255, 255,51), (255, 255,0), (230, 230,0)
    ]

    link_color = [
        # Body links
        0, 0, 0, 0, 7, 7, 7, 9, 9, 9, 9, 9, 16, 16, 16, 16, 16, 16, 16,
        # Face links
        4, 4, 4, 4, 4, 4, 4, 4,
        # Right Hand links (21)
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        # Left Hand links (21)
        6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6
    ]

    point_color = [
        # Body points
        16, 16, 16, 16, 16, 9, 9, 9, 9, 9, 9, 0, 0, 0, 0, 0, 0,
        # Face points
        4, 4, 4, 4, 4, 4, 4, 4,
        # Right Hand points (21)
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        # Left Hand points (21)
        6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6
    ]

    scale = resize / max(frame.shape[0], frame.shape[1])

    scores = keypoints[..., 2]
    keypoints = (keypoints[..., :2] * scale).astype(int)

    img = cv2.resize(frame, (0, 0), fx=scale, fy=scale)
    for kpts, score in zip(keypoints, scores):
        show = [0] * len(kpts)
        for (u, v), color in zip(skeleton, link_color):
            if score[u] > thr and score[v] > thr:
                cv2.line(img, kpts[u], tuple(kpts[v]), palette[color], 1,
                         cv2.LINE_AA)
                show[u] = show[v] = 1
        for kpt, show, color in zip(kpts, show, point_color):
            if show:
                cv2.circle(img, kpt, 1, palette[color], 2, cv2.LINE_AA)
    cv2.imwrite(filename, img)

In [ ]:
person_detector = Detector(
    model_path="/home/giakhang/dev/pose_sandbox/mmdeploy/rtmdet-trt/rtmdet-nano",
    device_name="cuda", 
    device_id=0
)

pose_detector = PoseDetector(model_path="mmdeploy/rtmpose-trt/rtmpose_l/rtmpose_l_both_sides",
    device_name="cuda",
    device_id=0)

[2024-11-13 14:29:02.239] [mmdeploy] [error] [model.cpp:26] File not found: "mmdeploy/rtmdet-trt/rtmdet_nano"
[2024-11-13 14:29:02.239] [mmdeploy] [error] [model.cpp:16] Failed to load model "mmdeploy/rtmdet-trt/rtmdet_nano"
[2024-11-13 14:29:02.245] [mmdeploy] [error] [model.cpp:21] failed to create model: file not exist (5) @ /home/giakhang/dev/pose_sandbox/mmdeploy/csrc/mmdeploy/core/model.cpp:27


RuntimeError: failed to create detector

In [29]:
img1 = cv2.imread("data/2024-10-19/2024-10-19-15:39/image/left/train/left_110.jpg")
img2 = cv2.imread("data/2024-10-19/2024-10-19-15:39/image/right/train/right_110.jpg")

batch = [img1, img2]

In [30]:
person_detection_rs = person_detector.batch(batch)

In [32]:
rs = pose_detector.batch(batch, [person_detection_rs[0][0][0, :4][None, :], person_detection_rs[1][0][0, :4][None, :]])

In [33]:
rs[0].shape

(1, 133, 3)